# Feature creation  

This notebook demonstrates the creation of features for the items that will be forecast using the machine learning model. 

In [1]:
# import the data
import pandas as pd
import random
import numpy as np
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
plt.rcParams['figure.figsize'] = (12, 5)
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings("ignore")

#Pandas option
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Let's read in the pickle dataframe
df_ml = pd.read_pickle("../data/df_ml.pkl")

In [3]:
df_ml.shape

(84448, 33)

### Imputing missing values + promotion features

In [4]:
# function to count missing values in dataframe
def count_missings(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = (df.isnull().sum() / df.isnull().count() * 100).sort_values(ascending = False)
    table = pd.concat([total, percent], axis = 1, keys = ["Total", "Percent"])
    table = table[table["Total"] > 0]
    return table

In [5]:
null_cols = count_missings(df_ml)
null_cols

,Total,Percent
UnitPromotionThreshold,78863,93.39
CommunicationChannel,78842,93.36
DiscountPercentage,78842,93.36
BasePrice,78842,93.36
SchoolHolidayNorth,58580,69.37
SchoolHolidaySouth,58580,69.37
SchoolHolidayMiddle,58580,69.37
UnitSales,1681,1.99
ShelfCapacity,728,0.86


The columns related to promotions have a high percentage of null's. Also the SchoolHoliday.... have quite a large number of nulls. There are a few null's in UnitSales and StoreCapacity. For UnitSales I do a simple imputation by taking the average of the sales for that item. For ShelfCapacity, use the avg shelf capacity for the number of stores present.

In [6]:
def impute_missing_values(df):
    """Imputes missing values for unitsales
        and ShelfCapacity. Drops AlcoholPercentage 
        and MinAge columns as they are all null
    Args:
        df (pd.DataFrame): Dataframe with sales data.
    Returns:
        pd.DataFrame: With unputed values
    """
    # create dict of number of stores vs avg shelf capacity
    # replace nan with values from dict by mapping StoreCount
    store_dict = dict(df.groupby('StoreCount')['ShelfCapacity'].mean())
    df.ShelfCapacity = df.ShelfCapacity.fillna(df.StoreCount.map(store_dict))
    
    # replace nan values for store unit sales with mean 
    # for that product/item
    sales_dict = dict(df.groupby('ItemNumber')['UnitSales'].mean())
    df.UnitSales = df.UnitSales.fillna(df.ItemNumber.map(sales_dict))
    
    # drop nan from unitsales as those items have no data
    df = df[df['UnitSales'].notna()]
    
    # drop AlcoholPercentage and MinAge columns 
    #df.drop(['MinAge', 'AlcoholPercentage'], axis=1, inplace=True)
    
    return df

For the promotion feature, since a large number of nulls are present, i.e. implying that most items are not on promotion on most days. We fill in the null values for these cols with zero as some ml models may not accept null values. Also we create a separate column showing that there was a null value for that feature in that row. The same process is followed for SchoolHolidays.

In [7]:
def promo_feats(df):
    """makes promo cols null values zero
    and adds extra col for rows with nan
    Args:
    df (pd.DataFrame): Dataframe with the sales data.
    Returns:
    Dataframe with numerically encoded data for cat cols
    """
    promo_list = ['UnitPromotionThreshold', 'DiscountPercentage', 'BasePrice', \
                  'SchoolHolidayNorth', 'SchoolHolidaySouth', 'SchoolHolidayMiddle']
    
    # If the values of promo cols is null then item was probably not
    # on promotion in the given day. Add this as a parameter.
    for promo_feat in promo_list:
        promo_col = "No" + promo_feat
        df[promo_col] = df[promo_feat].isna().astype(int)
        # Fill sell prices with zeros as not all models can handle
        # null values
        df[promo_feat] = df[promo_feat].fillna(0.0)
    
    return df

### Categorical features

The CategoryCode, Unitcode, ItemNumber and CommunicationChannel can be made into categorical features using an encoder. 

In [11]:
def transform_cat_feats(df):
    """makes null columns into unknown and cat columns
    are label encoded
    Args:
    df (pd.DataFrame): Dataframe with the sales data.
    Returns:
    Dataframe with numerically encoded data for cat cols
    """

    cat = [
        "ItemNumber",
        "GroupCode",
        "CategoryCode",
        "CommunicationChannel"    ]

    for feature in cat:
        encoder = LabelEncoder()
        df[feature] = encoder.fit_transform(df[feature])
        feat_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
        # save the mappings for later use
        fname = "../data/" + feature + ".pickle"
        with open(fname, 'wb') as handle:
            pickle.dump(feat_mapping, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    # convert T/F to 1/0 as some models might not accept T/F
    df['IsPromo'] = df['IsPromo'].astype(int)

    return df

### Time series features

The Date columns can be used to create time series features like day of week, month and year to capture seasonality. In addition, a lag and rolling window feature is very useful to capture recent information. To prevent leakage we create a lag and rolling window features using a 7 day shift (since we are predicting for 1 week ahead).

In [12]:
def create_ts_feats(df):
    """create time series features with
    day of week, month, year and weekend
    and lag + rolling mena features
    Args:
    df (pd.DataFrame): Dataframe with the sales data.
    Returns:
    Dataframe with numerically time series features
    """
    
    df['Date'] = pd.to_datetime(df['Date'])
    df['Weekday'] = df['Date'].dt.dayofweek
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df["IsWeekend"] = df["Weekday"].apply(
        lambda x: x in [5,6]).astype(int)
    
    dayLags = [7]
    lagSalesCols = [f"lag_{dayLag}" for dayLag in dayLags]
    for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
        df[lagSalesCol] = (
            df[["ItemNumber", "UnitSales"]].groupby("ItemNumber")["UnitSales"].shift(dayLag)
        )

    windows = [7, 28]
    for window in windows:
        for dayLag, lagSalesCol in zip(dayLags, lagSalesCols):
            df[f"rmean_{dayLag}_{window}"] = (
                df[["ItemNumber", lagSalesCol]]
                .groupby("ItemNumber")[lagSalesCol]
                .transform(lambda x: x.rolling(window).mean())
            )

    
    return df
    

Let's apply all these transformations and create the final dataframe for modeling.

In [13]:
df_final = impute_missing_values(df_ml)
df_final = transform_cat_feats(df_final)
df_final = promo_feats(df_final)
df_final = create_ts_feats(df_final)
df_final.head()

,StoreCount,ShelfCapacity,PromoShelfCapacity,UnitPromotionThreshold,IsPromo,national_holiday_new_years_day,national_holiday_good_friday,national_holiday_easter_sunday,national_holiday_easter_monday,national_holiday_kings_day,...,NoSchoolHolidayNorth,NoSchoolHolidaySouth,NoSchoolHolidayMiddle,Weekday,Month,Year,IsWeekend,lag_7,rmean_7_7,rmean_7_28
728,187,11712.00,0,0.00,0,0,0,0,0,0,...,1,1,1,0,1,2016,0,NaN,NaN,NaN
1210275,827,7145.00,0,0.00,0,0,0,0,0,0,...,1,1,1,0,1,2016,0,NaN,NaN,NaN
117976,832,8884.00,0,0.00,0,0,0,0,0,0,...,1,1,1,0,1,2016,0,NaN,NaN,NaN
1232151,11,110.00,0,0.00,0,0,0,0,0,0,...,1,1,1,0,1,2016,0,NaN,NaN,NaN
1246945,828,11180.00,0,0.00,0,0,0,0,0,0,...,1,1,1,0,1,2016,0,NaN,NaN,NaN


In [14]:
# We can drop the null rows from the rolling mean and
# lags as we haev enough data to cover on business cycle
df_final.dropna(inplace=True)

In [15]:
# Save dataframe to use later
df_final.to_pickle("../data/df_model.pkl")